In [1]:
using Interact
using LCMGL
using ForwardDiff
using RigidBodyDynamics
using DataFrames
using SimpleGradientDescent
using Ipopt

 in depwarn(::String, ::Symbol) at ./deprecated.jl:64
 in readall(::String, ::Vararg{String,N

}) at ./deprecated.jl:30
 in include_from_node1(::String) at ./loading.jl:488
 in __init__() at /home/rdeits/locomotion/explorations/point-cloud-signed-distance/packages/v0.5/Interact/src/Interact.jl:81
 in _include_from_serialized(::String) at ./loading.jl:150
 in _require_from_serialized(::Int64, ::Symbol, ::String, ::Bool) at ./loading.jl:187
 in _require_search_from_serialized(::Int64, ::Symbol, ::String, ::Bool) at ./loading.jl:217
 in require(::Symbol) at ./loading.jl:371
 in include_string(::String, ::String) at ./loading.jl:441
 in execute_request(::ZMQ.Socket, ::IJulia.Msg) at /home/rdeits/locomotion/explorations/point-cloud-signed-distance/packages/v0.5/IJulia/src/execute_request.jl:169
 in eventloop(::ZMQ.Socket) at /home/rdeits/locomotion/explorations/point-cloud-signed-distance/packages/v0.5/IJulia/src/eventloop.jl:8
 in (::IJulia.##9#15)() at ./task.jl:360
while loading /home/rdeits/locomotion/explorations/point-cloud-signed-distance/packages/v0.5/Interact/src/IJulia/setu

In [2]:
import Flash
reload("Flash")

In [3]:
squished = Flash.DepthData.read_point_cloud(open("data/squishable_squished_xyzrgb.txt"))

PointCloud with origin: [1.37689,0.775602,1.4848] containing 25164 points

In [4]:
Flash.DepthData.render_lcmgl(squished)

In [5]:
unsquished = Flash.DepthData.read_point_cloud(open("data/squishable_unsquished_xyzrgb.txt"))

PointCloud with origin: [1.38246,0.768824,1.48581] containing 25571 points

In [6]:
Flash.DepthData.render_lcmgl(unsquished)

In [7]:
model = Flash.Models.squishable()
state = Flash.ManipulatorState(model)
Flash.draw(state)

DrakeVisualizer.Visualizer(DrakeVisualizer.Robot(DrakeVisualizer.Link[DrakeVisualizer.Link(DrakeVisualizer.GeometryData[DrakeVisualizer.GeometryData{CoordinateTransformations.IdentityTransformation,GeometryTypes.HomogenousMesh{FixedSizeArrays.Point{3,Float64},GeometryTypes.Face{3,Int64,0},Void,Void,Void,Void,Void}}(HomogenousMesh(
    vertices: 302xFixedSizeArrays.Point{3,Float64},     faces: 600xGeometryTypes.Face{3,Int64,0}, )
,CoordinateTransformations.IdentityTransformation(),RGBA{Float64}(1.0,0.0,0.0,0.5))],"skin_1")]),1,PyLCM.LCM(PyObject <LCM object at 0x7f66981b62d0>))

In [8]:
function flatten(state::Flash.ManipulatorState)
    vcat(state.mechanism_state.q, state.deformation_data)
end

function unflatten!{T}(state::Flash.ManipulatorState, data::AbstractVector{T})
    nq = num_positions(state.mechanism_state)
    set_configuration!(state.mechanism_state, data[1:nq])
    state.deformation_data[:] = data[(nq + 1):end]
end

unflatten! (generic function with 1 method)

In [9]:
point_cloud = unsquished
Flash.DepthData.render_lcmgl(point_cloud)
sensed_points = [p.position for p in point_cloud.points]

cost = x -> begin
    typealias T eltype(x)
    state = Flash.ManipulatorState(model, T, T)
    unflatten!(state, x)
    Flash.draw(state, false)
    skin = Flash.skin(state)[1]
    raycast_cost = zero(T)
    for point in sensed_points
        raycast_cost += skin(point)^2
    end
#     raycast_cost = sum(point -> skin(point)^2, sensed_points)
    deformation_cost = zero(T)
    for ld in values(state.deformations)
        for d in ld
            deformation_cost += sum(d .^2)
        end
    end
    return raycast_cost / length(sensed_points) + 100*deformation_cost
end

# Use autodiff to prepare a function that will evaluate the gradient
# of that cost:
cost_and_gradient = x -> begin
    out = GradientResult(x)
    ForwardDiff.gradient!(out, cost, x)
    ForwardDiff.value(out), ForwardDiff.gradient(out)
end

trials = []


function test_gradient_descent()
    for i = 1:1
        xs = []

        # We'll initialize our estimate with a uniformly random pair
        # of joint angles
        x_estimated = flatten(Flash.ManipulatorState(model))
        x_estimated[5:7] = [.5, 0, .5]

        # Now we use gradient descent to recover the joint angles
        for i = 1:50
            push!(xs, x_estimated)

            # Evaluate the gradient of the point values w.r.t. the
            # joint angles:
            ci, gi = cost_and_gradient(x_estimated)

            if any(isnan, gi)
                @show x_estimated
                error("NaN in gradient")
            end
            
            # Descend the gradient
            step = -ci ./ gi
            step[1:7] = sign(step[1:7]) .* min.(0.01, abs.(gi[1:7]))
            step[8:end] = sign(step[8:end]) .* min.(1e-5, abs.(gi[8:end]))
            x_estimated += step

            # Descend the gradient
#             x_estimated -= 0.0000002 * gi
            
            # TODO: generalize this
            x_estimated[1:4] = x_estimated[1:4] / norm(x_estimated[1:4])
        end
        push!(trials, DataFrame(x=xs))
    end
end

test_gradient_descent (generic function with 1 method)

In [10]:
test_gradient_descent()
@time test_gradient_descent()
state = Flash.ManipulatorState(model)
unflatten!(state, trials[1][:x][end])
Flash.draw(state)

  8.305141 seconds (64.10 M allocations: 4.641 GB, 6.69% gc time)


DrakeVisualizer.Visualizer(DrakeVisualizer.Robot(DrakeVisualizer.Link[DrakeVisualizer.Link(DrakeVisualizer.GeometryData[DrakeVisualizer.GeometryData{CoordinateTransformations.IdentityTransformation,GeometryTypes.HomogenousMesh{FixedSizeArrays.Point{3,Float64},GeometryTypes.Face{3,Int64,0},Void,Void,Void,Void,Void}}(HomogenousMesh(
    vertices: 316xFixedSizeArrays.Point{3,Float64},     faces: 628xGeometryTypes.Face{3,Int64,0}, )
,CoordinateTransformations.IdentityTransformation(),RGBA{Float64}(1.0,0.0,0.0,0.5))],"skin_1")]),1,PyLCM.LCM(PyObject <LCM object at 0x7f66981b6300>))

In [11]:
x0 = flatten(Flash.ManipulatorState(model))
num_vars = length(x0)
solver = IpoptSolver(print_level=1, max_iter=20)
m = getmodel(GradientDescent(cost), num_vars, solver)
setwarmstart!(m, x0);

In [12]:
optimize!(m)
x = getsolution(m)
state = Flash.ManipulatorState(model)
unflatten!(state, x)
Flash.draw(state)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************



DrakeVisualizer.Visualizer(DrakeVisualizer.Robot(DrakeVisualizer.Link[DrakeVisualizer.Link(DrakeVisualizer.GeometryData[DrakeVisualizer.GeometryData{CoordinateTransformations.IdentityTransformation,GeometryTypes.HomogenousMesh{FixedSizeArrays.Point{3,Float64},GeometryTypes.Face{3,Int64,0},Void,Void,Void,Void,Void}}(HomogenousMesh(
    vertices: 280xFixedSizeArrays.Point{3,Float64},     faces: 556xGeometryTypes.Face{3,Int64,0}, )
,CoordinateTransformations.IdentityTransformation(),RGBA{Float64}(1.0,0.0,0.0,0.5))],"skin_1")]),1,PyLCM.LCM(PyObject <LCM object at 0x7f66981b6450>))